In [127]:
# Import important libraries of python
import pandas as pd
import numpy as np

In [128]:
# importing the opencv library
import cv2

### Methods

In [124]:

# Open the video file named 'assignmnet_video.mpg' using cv2.VideoCapture()
video_capture = cv2.VideoCapture('assignmnet_video.mpg')

# Read the first frame of the video
success, frame = video_capture.read()

# Convert the read frame from color to grayscale
gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
width, height = gray_frame.shape

# Initialize foreground and background frames
foreground = np.zeros((width,height,3), dtype=np.uint8)
background = np.zeros((width,height,3), dtype=np.uint8)

# Parameters
background_ratio = 0.01  # Determine the proportion of data to be accounted for background
learning_rate = 0.0001  # Set the learning rate for updating parameters
second_learning_rate = 0.1  # Set the second learning rate
num_frames = 10 # Define the total number of frames


In [125]:
# Define parameters for Gaussian Mixture Models (GMMs):

# For each pixel, we assume three Gaussian distributions (K=3) with independence and equal variance for each color.

# Initialize the mean of each Gaussian for every pixel, represented as [Mean1, Mean2, Mean3] in RGB format.
gaussian_mean1 = np.zeros((width,height,3), dtype=np.float64)
gaussian_mean2 = np.zeros((width,height,3), dtype=np.float64)
gaussian_mean3 = np.zeros((width,height,3), dtype=np.float64)

# Initialize the covariance matrix for each Gaussian distribution.
gaussian_covariance = np.ones((width,height,3), dtype=np.float64)

# Initialize the weight of each Gaussian distribution for each pixel, with each value set to one.
gaussian_weight = np.ones((width,height,3), dtype=np.float64)


In [126]:
# Iterate over each frame in the video up to a specified number of frames (Iteration)
count = 0
while count <= num_frames:
    # Read the next frame from the video
    ret, frame = file.read()
    
    # Calculate the sum of weights for each pixel
    wsum = 0
    
    # Iterate over each pixel in the frame
    for a in range(width):
        for b in range(height):
            # Initialization of Parameters and Distribution Values
            if count == 0:
                # Initialize mean, covariance, and weight values for each Gaussian distribution
                gaussian_mean1[a, b, :] = [1, 2, 3]
                gaussian_mean2[a, b, :] = [1, 2, 3]
                gaussian_mean3[a, b, :] = [1, 2, 3]
                gaussian_covariance[a, b, :] = [2, 2, 2]
                gaussian_weight[a, b, :] = [0.33, 0.33, 0.34]

            # Initialize an array to store distances and set match to -1
            dist = np.zeros((2, 3), dtype=np.float64)
            match = -1

            # Iterate over each Gaussian distribution
            for i in range(0, 3):
                # Calculate Mahalanobis distance and threshold for the current pixel and Gaussian
                temp = frame[a, b, :] - [gaussian_mean1[a, b, i], gaussian_mean2[a, b, i], gaussian_mean3[a, b, i]]
                euclid_distance = np.dot(temp, temp)
                threshold = (1 / gaussian_covariance[a, b, i]) * euclid_distance

                # Check if the threshold is below a certain value to determine a match
                if threshold < (2.5 * 2.5):
                    match = i

                    # Update mean, covariance, and weight for the matched distribution
                    gaussian_mean1[a, b, i] = (1 - rho) * gaussian_mean1[a, b, i] + rho * frame[a, b, 0]
                    gaussian_mean2[a, b, i] = (1 - rho) * gaussian_mean2[a, b, i] + rho * frame[a, b, 1]
                    gaussian_mean3[a, b, i] = (1 - rho) * gaussian_mean3[a, b, i] + rho * frame[a, b, 2]
                    gaussian_covariance[a, b, i] = (1 - rho) * gaussian_covariance[a, b, i] + (rho * euclid_distance)
                    gaussian_weight[a, b, i] = (1 - alpha) * gaussian_weight[a, b, i] + alpha
                    wsum += gaussian_weight[a, b, i]
                else:
                    # Update weight for non-matched distributions
                    gaussian_weight[a, b, i] = (1 - alpha) * gaussian_weight[a, b, i]
                    wsum += gaussian_weight[a, b, i]

            # Normalize weights
            gaussian_weight[a, b, :] = (1/wsum)*gaussian_weight[a, b, :]

            # Calculate distances and sort the array
            for i in range(3):
                dist[0, i], dist[1, i] = i, (gaussian_weight[a, b, i] / math.sqrt(gaussian_covariance[a, b, i]))
            for i in range(3):
                for j in range(0, 2 - i):
                    if dist[1, j] < dist[1, j + 1]:
                       dist[0, j + 1], dist[0, j] = dist[0, j], dist[0, j + 1]
                       dist[1, j + 1], dist[1, j] = dist[1, j], dist[1, j + 1]

            # Find the most and least probable distributions
            most_prob = int(dist[0, 0])
            least_prob = int(dist[0, 2])

            # If no match found, update parameters for the least probable distribution
            if match == -1:
                wsum -= gaussian_weight[a, b, least_prob]
                gaussian_mean1[a, b, least_prob] = frame[a, b, 0]
                gaussian_mean2[a, b, least_prob] = frame[a, b, 1]
                gaussian_mean3[a, b, least_prob] = frame[a, b, 2]
                gaussian_covariance[a, b, least_prob] = 20
                gaussian_weight[a, b, least_prob] = 0.1
                wsum += gaussian_weight[a, b, least_prob]
                gaussian_weight[a, b, :] = (1/wsum)*gaussian_weight[a, b, :]
        
            # Calculate mean values for background

            # Initialize variables to keep track of the sum of weights, mean values for each color channel, and the number of Gaussian distributions considered
            B = 0  # Counter for the number of Gaussian distributions considered
            sum_threshold = np.float64(0)  # Sum of weights of the Gaussian distributions
            mean_bg1, mean_bg2, mean_bg3 = np.float64(0), np.float64(0), np.float64(0)  # Initialize mean values for each color channel

            # Iterate over the three Gaussian distributions
            for i in range(3):
                # Accumulate the sum of weights of the Gaussian distributions
                sum_threshold += gaussian_weight[a, b, int(dist[0, i])]
    
                # Calculate the weighted sum of mean values for each color channel based on the Gaussian distributions
                mean_bg1 += gaussian_weight[a, b, int(dist[0, i])] * gaussian_mean1[a, b, int(dist[0, i])]
                mean_bg2 += gaussian_weight[a, b, int(dist[0, i])] * gaussian_mean2[a, b, int(dist[0, i])]
                mean_bg3 += gaussian_weight[a, b, int(dist[0, i])] * gaussian_mean3[a, b, int(dist[0, i])]
    
                # Increment the counter for the number of Gaussian distributions considered
                B += 1
    
                # Check if the sum of weights exceeds the threshold value T
                if sum_threshold > T:
                    # Normalize the mean values by dividing them by the sum of weights
                    mean_bg1 = mean_bg1 / sum_threshold
                    mean_bg2 = mean_bg2 / sum_threshold
                    mean_bg3 = mean_bg3 / sum_threshold
                    # Exit the loop since the threshold condition is met
                    break


            # Set the background values based on most probable distributions
            bg[a, b :] = [mean_bg1, mean_bg2, mean_bg3]
            
            # Set foreground values based on match and distribution thresholds
            if match == -1:
                fg[a, b, :] =  [255, 255, 255]
            else:
                for i in range(B):
                    if match == dist[0, i]:
                        fg[a, b, :] = [0, 0, 0]
                        break
                    else:
                        fg[a, b, :] = [255, 255, 255]

    # Save the background frame and median filtered foreground frame
    cv2.imwrite("Background%d.jpg" % count, bg)
    med = cv2.medianBlur(fg, 3)
    cv2.imwrite("Foreground%d.jpg" % count, med)

    # Increment the count
    count += 1

    # Check for the exit condition
    if cv2.waitKey(1000) == 27 & 0xFF:
        break


### Convert into video 

In [88]:
# Create video writers for foreground and background videos
foreground_video = cv2.VideoWriter('Foreground.mp4', cv2.VideoWriter_fourcc(*'MP4V'), 30, (width, height))
background_video = cv2.VideoWriter('Background.mp4', cv2.VideoWriter_fourcc(*'MP4V'), 30, (width, height))

# Write foreground frames to video
count = 0
while count <= Iteration:
    # Read foreground frame
    foreground_frame = cv2.imread(r"kunal%d.jpg" % count, 1)
    # Resize frame to match dimensions
    resized_foreground_frame = cv2.resize(foreground_frame, (width, height))
    # Write resized foreground frame to video
    foreground_video.write(resized_foreground_frame)
    # Break loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    count += 1

# Write background frames to video
count = 0
while count <= Iteration:
    # Read background frame
    background_frame = cv2.imread(r"anand%d.jpg" % count, 1)
    # Resize frame to match dimensions
    resized_background_frame = cv2.resize(background_frame, (width, height))
    # Write resized background frame to video
    background_video.write(resized_background_frame)
    # Break loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    count += 1

# Release video capture and writers, close all windows
capture.release()
foreground_video.release()
background_video.release()
cv2.destroyAllWindows()


OpenCV: FFMPEG: tag 0x5634504d/'MP4V' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'
OpenCV: FFMPEG: tag 0x5634504d/'MP4V' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'
